# Overview

Ref: [Calculating centred and non-centred volatility](http://vixandmore.blogspot.com/2009/12/calculating-centered-and-non-centered.html)

Stock Volatility measures how much a stock tends to move. There are many ways to calculate volatility, for e.g. through:

- Daily/Weekly/Monthly range
- Average True Range
- Standard Deviation

Standard Deviation is the most popular way to measure volatility. Standard Deviation for stock price can be computed in multiple ways. Let us go through them.

In [ ]:
import pandas as pd
import math

# computing SD of a series of values
data = [161.47, 159.86, 159.27, 159.98, 159.78, 157.21, 157.5, 157.86, 160.95, 161.6, 159.85, 
         157.48, 155.32, 160.43, 159.45, 158.19, 155.78, 154.96, 156.53, 149.46, 148.15] # gives sd of 1.59%

s = pd.Series(data, name='data')
df = pd.DataFrame(s)

## 1) From math library's standard deviation computation.

The code for this is as follows:


In [ ]:
asd = df.data.std()
amean = df.data.mean()
dict(asd=asd, amean=amean)

Arithmetic standard deviation makes sense for a stable mean. However, in stock prices the price keeps current price keeps changing. So it is better to use **geometric standard deviation** or GSD. 

Here are different ways of getting GSD:

## 2a) From daily log returns assuming zero mean and averaging squares
Ref: [this Quora link](https://www.quora.com/How-do-you-calculate-the-standard-deviation-for-a-stock)

Formula for geometric standard deviation is represented by:
     
gsd1 = $\exp{\Big(\sqrt{{\frac{\sum_{1}^n (\ln\frac{x[n]}{x[n-1]})^{2}}{n}}}}\Big)$

Geometric standard deviation is computed over geometric mean, which is represented by the formula:

gm = $\left (\prod_{a=1}^{b}x_i  \right )^{\frac{1}{n}} = \sqrt[n]{x_1x_2...x_n}$

The upper and lower values of 1 GSD are computed by dividing and multiplying geometric mean with the geometric standard deviation

The code for these are as follows:

```
import math
from scipy.stats.mstats import gmean
sdmult = 2 # no of standard deviations
gsd1 = math.exp(math.sqrt(df.data.rolling(2).apply(lambda x: math.log(x[0]/x[-1])**2, raw=True).mean()))
gm = gmean(df)
glo1 = gm*(1-(gm-gm/gsd1)/gm*sdmult) # gm/gsd1 ... but taking sdmult into consideration
ghi1 = gm*(1+(gm*gsd1-gm)/gm*sdmult) # gm*gsd1 ... but taking sdmult into consideration

```



In [ ]:
import math
from scipy.stats.mstats import gmean
sdmult = 2 # no of standard deviations
gsd1 = math.exp(math.sqrt(df.data.rolling(2).apply(lambda x: math.log(x[0]/x[-1])**2, raw=True).mean()))
gm = gmean(df)
glo1 = gm*(1-(gm-gm/gsd1)/gm*sdmult) # gm/gsd1 ... but taking sdmult into consideration
ghi1 = gm*(1+(gm*gsd1-gm)/gm*sdmult) # gm*gsd1 ... but taking sdmult into consideration

dict(gm=gm, gsd1=gsd1, glo1=glo1, ghi1=ghi1)

## 2b) From daily log returns without squaring and square-rooting

The formula for this is:

sd2 = ${\exp(\sigma\Big({{\ln(\frac{x[n]}{x[n-1]})}}\Big))}$

This equation gives a slightly lesser band than taking rooting...

In [ ]:
import math
from scipy.stats.mstats import gmean
sdmult = 2 # no of standard deviations
gsd2 = math.exp(df.data.rolling(2).apply(lambda x: math.log(x[0]/x[-1]), raw=True).std())
glo2 = gm*(1-(gm-gm/gsd2)/gm*sdmult) # gm/gsd2 ... but taking sdmult into consideration
ghi2 = gm*(1+(gm*gsd2-gm)/gm*sdmult) # gm*gsd2 ... but taking sdmult into consideration

dict(gm=gm, gsd2=gsd2, glo2=glo2, ghi2=ghi2)

## 2c) Centered Historical Volatiity using look-back (Bill Luby)

The steps are as follows:

1. Select a desired lookback period in trading days (lookback period)
2. Gather closing prices for the full lookback period, plus one additional day (lookback +1)
4. Calculate the daily close-to-close price changes in a security for each day in the lookback period (daily change)
5. Determine the natural log of each daily percentage change (log of daily changes)
6. Calculate the mean of all the natural logs of the closing prices for the lookback period (log lookback mean)
7. For each day, subtract the lookback mean from the log of daily changes (daily difference)
8. Square all the differences between the mean and the daily change (daily variance)
9. Sum all the squares of the differences (sum of variances)
10. Divide the sum of the squares of the variances by the lookback period (lookback variance)
11. Take the square root of the lookback variance (historical volatility, expressed as a standard deviation)
12. To convert this to an annual volatility percent, take HV expressed as standard deviation and multiply it by square root of no of trading days in a year (252) and then by 100.
13. Compute the high and low bands for the lookback period

## 2d) Non-centered Historical volatility on a lookback (Bill Luby - faster)

The previous calculation reflects a centred approach where daily price changes are characterized relative to a mean value for the entire period. Instead of that one could assume that, in the long run, the mean change in price approaches zero and hence not meamingful. So, if the mean is not meaningful, we can dump it and not subtract it from the daily changes, so all calculations involving the mean can be dropped! This non-centred approach for historical volaility is sometimes called "ditching the mean".

This method is shorter and [gives better numbers to traders.](http://vixandmore.blogspot.com/2009/12/calculating-centered-and-non-centered.html). 

In this calculation the following steps are taken:

1. Select a desired (10-day) lookback period in trading days (lookback period)
2. Gather closing prices for the full lookback period, plus one additional day (lookback +1)
4. Calculate the daily close-to-close price changes in a security for each day in the lookback period (daily change)
5. Determine the natural log of each daily percentage change (log of daily changes)
6. Determine the historical volatility by multiplying the Standard Deviation of the log change with root of 252 and multiplying by 100.

Let us now see how all of the above work on a pandas ohlc data-set




In [17]:
import pandas as pd
import numpy as np
import os
import math
from pathlib import Path

MARKET = 'nse'
TRADING_DAYS=252
STDMULT = 2

oldp = str(Path(os.getcwd()).parents[3])+f"\ibkr\data\{MARKET.lower()}"
newp = str(Path(os.getcwd()).parents[0])+f"\ib\data\{MARKET.lower()}"

In [19]:
df_all_ohlc = pd.read_pickle(newp+'\\_df_ohlc.pkl')
df_und = pd.read_pickle(newp+'\\_df_und.pkl')

# pick up NIFTY50 only and reset its index
cols = ['symbol', 'dte', 'date', 'close', 'rise', 'fall'] # interested columns
df = df_all_ohlc[df_all_ohlc.symbol=='NIFTY50']

# extract from df_und data
df = df.set_index('symbol').join(df_und[['symbol', 'undPrice', 'iv_ib', 'hv_ib', 'avg_div']].set_index('symbol')).reset_index()

# replace iv_ib and hv_ib as per dte
df = df.assign(
    iv_ib=df.iv_ib*[math.sqrt(j/365) for j in df.index],
    hv_ib=df.hv_ib*[math.sqrt(j/365) for j in df.index])

Here are the comparisons of various algorithms for hv.

### 1) Luby's formula


In [ ]:
def hv_luby(df):
    """Computes historical volatility based on Bill Luby's formula
    Ref: http://vixandmore.blogspot.com/2009/12/calculating-centered-and-non-centered.html
    Arg: df as dataframe for a symbol with close field
    Outputs: historical volaility percentage
    """
    # Determine the natural log of each daily percentage change (log of daily changes)
    log_of_changes = df.close.rolling(2).apply(
        func=lambda x: math.log(x[1]/x[0]),
        raw=True)

    # Calculate the mean of all the natural logs of the closing prices for the lookback period (log lookback mean)
    lookback_mean = log_of_changes.expanding().mean()

    # For each day, subtract the lookback mean from the log of daily changes (daily difference)
    daily_difference = log_of_changes - lookback_mean

    # Square all the differences between the mean and the daily change (daily variance)
    daily_variance = daily_difference**2

    # Sum all the squares of the differences (sum of variances)
    sum_of_variances = daily_variance.expanding().sum()

    # Divide the sum of the squares of the variances by the lookback period (lookback variance)
    lookback_variance = sum_of_variances/sum_of_variances.index

    # Take the square root of the lookback variance (historical volatility, expressed as a standard deviation)
    hv_as_sd = lookback_variance.apply(lambda x: math.sqrt(x))

    # Compute annual volatility by multiplying by root of 252
    hv_luby = hv_as_sd.apply(lambda x: x*math.sqrt(TRADING_DAYS))

    df = df.assign(hv_luby=hv_luby)

    return df

df  = hv_luby(df)

### 2) TastyTrades formula

In [ ]:
# Determine the natural log of each daily percentage change (log of daily changes)
log_of_changes = df.close.rolling(2).apply(
    func=lambda x: math.log(x[1]/x[0]),
    raw=True)

# Calculate the mean of all the natural logs of the closing prices for the lookback period (log lookback mean)
lookback_mean = log_of_changes.expanding().mean()

# For each day, subtract the lookback mean from the log of daily changes (daily difference)
daily_difference = log_of_changes - lookback_mean

# Square all the differences between the mean and the daily change (daily variance)
daily_variance = daily_difference**2

# Sum all the squares of the differences (sum of variances)
sum_of_variances = daily_variance.expanding().sum()

# Divide the sum of the squares of the variances by the lookback period (lookback variance)
lookback_variance = sum_of_variances/sum_of_variances.index

# Take the square root of the lookback variance (historical volatility, expressed as a standard deviation)
hv_as_sd = lookback_variance.apply(lambda x: math.sqrt(x))

# Compute annual volatility by multiplying by root of 252
hv_luby = hv_as_sd.apply(lambda x: x*math.sqrt(TRADING_DAYS))

In [ ]:
hv_as_sd*math.sqrt(TRADING_DAYS)

In [ ]:
# Let us see what the Luby's standard deviation for the entire data set is:

def hv_luby_365(df):
    """Compute Luby's historical volatility for 365 day chunks
    Arg: df as dataframe of 365 days
    Returns: historical volatilty as a float"""
    df.reset_index(drop=True)
    log_of_changes = df.close.rolling(2).apply(lambda x: math.log(x[0]/x[-1]))
    lookback_mean = log_of_changes.expanding().mean()
    daily_difference = log_of_changes - lookback_mean
    daily_variance = daily_difference**2
    sum_of_variances = daily_variance.sum()
    lookback_variance = sum_of_variances/(len(df)+1)
    hv_as_sd = math.sqrt(lookback_variance)
    hv_luby = hv_as_sd*math.sqrt(TRADING_DAYS)
    return hv_luby

# df.apply(lambda x: x[::-1]).apply(hv_luby_365)
hv_luby_365(df[:365])

dfs = [df[x: x+365] for x in range(0, len(df)-364)]
[hv_luby_365(df) for df in dfs]

In [ ]:
def hv_tw(df):
    """Historical Volatility by Tasty Works
    Ref: https://youtu.be/omVKR85pw2s
    Arg: df as dataframe for a symbol with close field
    Outputs: historical volaility percentage
    Note: To compute price bands
    Upper Band = price*(1 + ln_avg + STDMULT*ln_stdev)
    Lower Band = price*(1 + ln_avg - STDMULT*ln_stdev)
    """
    # Log-normal of price ratio
    ln = df.close.rolling(2).apply(
            lambda x: math.log(x[1]/x[0]))

    # stdev and mean of ln
    ln_avg = ln.expanding().mean()
    ln_stdev = ln.expanding().std(ddof=1)

    df = df.assign(ln_avg=ln_avg, ln_stdev=ln_stdev)

    return df

df = hv_tw(df)

In [ ]:
# Putting all of them together

up_tw = pd.Series(df.undPrice*(1+df.ln_avg+STDMULT*df.ln_stdev), name='up_tw')
lo_tw = pd.Series(df.undPrice*(1+df.ln_avg-STDMULT*df.ln_stdev), name='lo_tw')
up_luby = pd.Series(df.undPrice*(1+STDMULT*df.hv_luby), name='up_luby')
lo_luby = pd.Series(df.undPrice*(1-STDMULT*df.hv_luby), name='lo_luby')
up_iv_ib = pd.Series(df.undPrice*(1+STDMULT*df.iv_ib), name='up_iv_ib')
lo_iv_ib = pd.Series(df.undPrice*(1-STDMULT*df.iv_ib), name='lo_iv_ib')
up_hv_ib = pd.Series(df.undPrice*(1+STDMULT*df.hv_ib), name='up_hv_ib')
lo_hv_ib = pd.Series(df.undPrice*(1-STDMULT*df.hv_ib), name='lo_hv_ib')

df=df.assign(up_tw=up_tw, lo_tw=lo_tw, up_luby=up_luby, lo_luby=lo_luby, up_iv_ib=up_iv_ib, lo_iv_ib=lo_iv_ib, up_hv_ib=up_hv_ib, lo_hv_ib=lo_hv_ib)

<em>Note</em>: The following cells are work-in-progress and have been temporarily abandoned

In [23]:
from scipy.stats.mstats import gmean

# Get square of log ratios
log_square = df.close.rolling(2).apply(
    lambda x: math.log(x[0]/x[-1])**2,
    raw=True)

# Sum the squares
sum_log_square = log_square.expanding().sum()

# Divide the sum with number of observations
avg_sum_log_square = sum_log_square/sum_log_square.index

# get the root of avg sum log square
gsd_fact = avg_sum_log_square.apply(math.sqrt).apply(math.exp)-1

# get the geometric mean
gm = df.close.expanding().apply(gmean)

# get the geometric mean difference from undPrice
gm_fact = 1-gm/df.undPrice.unique()

df = df.assign(gm_fact = gm_fact, gsd_fact = gsd_fact)

# what we do with these gm and gsdd factors are unclear, especially with negative gm_facts
df

,symbol,dte,date,open,high,low,close,volume,average,barCount,rise,fall,undPrice,iv_ib,hv_ib,avg_div,gm_fact,gsd_fact
0,NIFTY50,0,2020-02-18,12027.85,12028.95,11960.50,11964.85,0,0.0,1285,0.00,0.00,11966.049805,0.000000,0.000000,148.6915,0.000100,NaN
1,NIFTY50,1,2020-02-17,12131.55,12159.35,12037.20,12045.80,0,0.0,21269,569.40,318.45,11966.049805,0.007247,0.007353,148.6915,-0.003277,0.006766
2,NIFTY50,2,2020-02-14,12194.70,12246.50,12091.65,12113.45,0,0.0,21310,895.40,541.80,11966.049805,0.010249,0.010398,148.6915,-0.006281,0.006217
3,NIFTY50,3,2020-02-13,12219.55,12219.70,12140.00,12174.65,0,0.0,21131,883.40,691.85,11966.049805,0.012552,0.012735,148.6915,-0.009058,0.005855
4,NIFTY50,4,2020-02-12,12151.00,12231.55,12145.05,12201.20,0,0.0,20989,782.60,660.25,11966.049805,0.014494,0.014705,148.6915,-0.011168,0.005185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,NIFTY50,486,2018-02-23,10405.00,10499.00,10397.25,10491.05,0,0.0,20708,1822.80,1473.80,11966.049805,0.159765,0.162090,148.6915,0.065673,0.008564
487,NIFTY50,487,2018-02-22,10354.35,10397.45,10341.55,10382.70,0,0.0,20572,1796.25,1582.15,11966.049805,0.159930,0.162256,148.6915,0.065815,0.008569
488,NIFTY50,488,2018-02-21,10426.00,10426.00,10349.80,10397.45,0,0.0,20628,1735.05,1567.40,11966.049805,0.160094,0.162423,148.6915,0.065953,0.008560
489,NIFTY50,489,2018-02-20,10391.00,10428.70,10348.20,10360.40,0,0.0,20933,1667.40,1604.45,11966.049805,0.160258,0.162589,148.6915,0.066098,0.008553


In [24]:
# this was made to test and build a case for computation with geometric mean
# Failed Again
# Refer to https://stats.stackexchange.com/questions/449482/geometric-standard-deviation-on-a-shifted-mean for stackoverflow question raised

import numpy as np
import pandas as pd
import math

from scipy.stats.mstats import gmean
from scipy.stats import gstd

np.random.seed(999)

bmark = 10 # benchmark
sdmult = 1 # standard deviation multiple

data = pd.Series(np.random.randint(8, high=12, size=100), name='Value')

am = pd.Series(data.expanding().mean(), name="aMean") # arithmetic mean
asd = pd.Series(data.expanding().std(ddof=1), name="aSD") # arithmetic standard deviation
a_abv = pd.Series(bmark-(bmark-am)+sdmult*asd, name="a_abv") # benchmark value above multiple of sd
a_blw = pd.Series(bmark-(bmark-am)-sdmult*asd, name="a_blw") # benchmark value below multiple of sd

gm_fact = pd.Series(data.expanding().apply(gmean), name="gMean") # geometric mean factor
gsd_fact = pd.Series(data.apply(math.log).expanding().apply(np.std).apply(math.exp), name="gSD") # geometric standard deviation factor

g_ucl = pd.Series((bmark-(bmark-gm_fact))*(math.sqrt(sdmult)+gsd_fact), name="g_ucl") # geometric upper control limit
g_lcl = pd.Series((bmark-(bmark-gm_fact))/(math.sqrt(sdmult)+gsd_fact), name="g_lcl") # geometric lower control limit

# computing above and below benchmark
pd.DataFrame([data, am, asd, a_abv, a_blw, gm_fact, gsd_fact, g_ucl, g_lcl]).T

,Value,aMean,aSD,a_abv,a_blw,gMean,gSD,g_ucl,g_lcl
0,8.0,8.000000,NaN,NaN,NaN,8.000000,1.000000,16.000000,4.000000
1,8.0,8.000000,0.000000,8.000000,8.000000,8.000000,1.000000,16.000000,4.000000
2,9.0,8.333333,0.577350,8.910684,7.755983,8.320335,1.057094,17.115710,4.044704
3,9.0,8.500000,0.577350,9.077350,7.922650,8.485281,1.060660,17.485281,4.117749
4,8.0,8.400000,0.547723,8.947723,7.852277,8.385925,1.059399,17.269966,4.072026
...,...,...,...,...,...,...,...,...,...
95,11.0,9.520833,1.169608,10.690441,8.351226,9.448998,1.131470,20.140256,4.433090
96,11.0,9.536082,1.173153,10.709236,8.362929,9.463815,1.131829,20.175233,4.439295
97,10.0,9.540816,1.168031,10.708847,8.372785,9.469139,1.131253,20.181128,4.442992
98,8.0,9.525253,1.172329,10.697582,8.352923,9.453027,1.131850,20.152440,4.434188
